In [17]:
!pip install verovio streamlit tiktoken

In [18]:
%%writefile backend.py
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer

class OCRUsingGOT:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained('ucaslcl/GOT-OCR2_0', trust_remote_code=True)
        self.model = AutoModel.from_pretrained('ucaslcl/GOT-OCR2_0', trust_remote_code=True,
                               device_map = 'cuda', use_safetensors=True,
                               pad_token_id = self.tokenizer.eos_token_id)
        self.model.eval().cuda()

    def extract_text(self, image):



        extracted_text = ''
        with torch.no_grad():
            extracted_text = self.model.chat(self.tokenizer, image, ocr_type = 'ocr')

        return extracted_text


Overwriting backend.py


In [25]:
%%writefile app.py
import streamlit as st
from PIL import Image
import tempfile
import os
from backend import OCRUsingGOT

st.set_page_config(
    page_title="OCR Using GOT",
    layout="centered",
    initial_sidebar_state="auto",
)

@st.cache_resource
def initialize_ocr():
    return OCRUsingGOT()

ocr_extractor = initialize_ocr()

st.title("🖼️ OCR Using GOT")
st.write("Upload an image, and the model will extract the text for you.")

image_file = st.file_uploader("Upload an image", type=['png', 'jpg', 'jpeg'])

if image_file is not None:
    try:
        pil_image = Image.open(image_file).convert('RGB') 
        st.image(pil_image, caption='Uploaded Image', use_column_width=True)

        with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as tmp_file:
            pil_image.save(tmp_file.name)
            temp_file_path = tmp_file.name

        with st.spinner('Extracting text from the image...'):
            extracted_text = ocr_extractor.extract_text(temp_file_path)  

        st.subheader("📝 Extracted Text:")
        if extracted_text:
            st.write(extracted_text)
            
            search_term = st.text_input("Search for text:", "")
            if search_term:
                highlighted_text = extracted_text.replace(
                    search_term, f'<span style="background-color: rgba(0, 255, 0, 0.3);">{search_term}</span>'
                )
                st.markdown(highlighted_text, unsafe_allow_html=True)  
            else:
                st.write("No search term provided.")
        else:
            st.write("No text found in the image.")

    except Exception as e:
        st.error(f"An error occurred while processing the image: {e}")
    finally:
        if 'temp_file_path' in locals() and os.path.exists(temp_file_path):
            os.remove(temp_file_path)


Overwriting app.py


In [ ]:
!curl ipv4.icanhazip.com


In [22]:
!npm install localtunnel

m#################.] - reify:yargs-parser: http fetch GET 200 https://registry.
added 22 packages in 2s

3 packages are looking for funding
  run `npm fund` for details
npm notice 
npm notice New minor version of npm available! 10.5.0 -> 10.8.3
npm notice Changelog: https://github.com/npm/cli/releases/tag/v10.8.3
npm notice Run npm install -g npm@10.8.3 to update!
npm notice 


In [ ]:
!streamlit run app.py &>./logs.txt & npx localtunnel --port 8501

your url is: https://great-weeks-clap.loca.lt
